In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['CONFIG_DIR'] = "/content/gdrive/My Drive/Project"
%cd /content/gdrive/My Drive/Project
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Project
kitti/  output/  readme.txt


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.model_selection import train_test_split
import gc; gc.enable() # memory is tight

from keras import layers, models
from keras import backend as K
from keras.optimizers import Adam

from google.colab.patches import cv2_imshow

In [ ]:
# img_dir = 'kitti dataset/'
# ../input/kitti_single/training/label_2/
train_image_dir_l = 'kitti/training/label_2/'
# label_test_image_dir = os.path.join(os.getcwd(), 'streets\\test\\labels\\')

train_image_dir = 'kitti/training/image_2/'
# test_image_dir = os.path.join(os.getcwd(), 'streets\\test\\images\\')

images =  [(train_image_dir+f) for f in listdir(train_image_dir) if isfile(join(train_image_dir, f))]
masks = [(train_image_dir_l+f) for f in listdir(train_image_dir_l) if isfile(join(train_image_dir_l, f))]

df = pd.DataFrame(np.column_stack([images, masks]), columns=['images', 'masks'])

In [ ]:
df1 = df.sort_values(by='images')['images'].reset_index()
# df1 = df.sort_values(by='a')['a']
df2 = df.sort_values(by='masks')['masks'].reset_index()
# df2 = df.sort_values(by='b')['b']
df['images'] = df1['images']
df['masks'] = df2['masks']
del df1, df2

In [ ]:
df

,images,masks
0,kitti/training/image_2/000000.png,kitti/training/label_2/000000.txt
1,kitti/training/image_2/000001.png,kitti/training/label_2/000001.txt
2,kitti/training/image_2/000002.png,kitti/training/label_2/000002.txt
3,kitti/training/image_2/000003.png,kitti/training/label_2/000003.txt
4,kitti/training/image_2/000004.png,kitti/training/label_2/000004.txt
...,...,...
7476,kitti/training/image_2/007476.png,kitti/training/label_2/007476.txt
7477,kitti/training/image_2/007477.png,kitti/training/label_2/007477.txt
7478,kitti/training/image_2/007478.png,kitti/training/label_2/007478.txt
7479,kitti/training/image_2/007479.png,kitti/training/label_2/007479.txt


In [ ]:

LABEL_COLORS = {
    'Car': (255,0,0), 
    'Van': (255,255,0), 
    'Truck': (255,255,255),
    'Pedestrian': (0,255,255),
    'Person_sitting': (0,255,255), 
    'Cyclist': (0,128,255), 
    'Tram': (128,0,0),
    'Misc': (0,255,255),
    'DontCare': (255,255,0)
}




In [ ]:
def create_mask_car(mask_dir, img_shape):
    mask = np.zeros(shape=(img_shape[0], img_shape[1], 1))

    with open(mask_dir) as f:
        content = f.readlines()
    content = [x.split() for x in content] 
    for item in content:
        if item[0]=='Car' :
            ul_col, ul_row = int(float(item[4])), int(float(item[5]))
            lr_col, lr_row = int(float(item[6])), int(float(item[7]))
            
            mask[ul_row:lr_row, ul_col:lr_col, 0] = 1 
    return mask


In [ ]:
def create_mask_pedestrian(mask_dir, img_shape):
    mask = np.zeros(shape=(img_shape[0], img_shape[1], 1))

    with open(mask_dir) as f:
        content = f.readlines()
    content = [x.split() for x in content] 
    for item in content:
        if item[0]=='Car' :
            ul_col, ul_row = int(float(item[4])), int(float(item[5]))
            lr_col, lr_row = int(float(item[6])), int(float(item[7]))
            
            mask[ul_row:lr_row, ul_col:lr_col, 0] = 1 
    return mask

In [ ]:
def create_mask_van(mask_dir, img_shape):
    mask = np.zeros(shape=(img_shape[0], img_shape[1], 1))

    with open(mask_dir) as f:
        content = f.readlines()
    content = [x.split() for x in content] 
    for item in content:
        if item[0]=='Van' :
            ul_col, ul_row = int(float(item[4])), int(float(item[5]))
            lr_col, lr_row = int(float(item[6])), int(float(item[7]))
            
            mask[ul_row:lr_row, ul_col:lr_col, 0] = 1 
    return mask

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.25, shuffle=False)
resized_shape = (160, 256)  

In [ ]:
BATCH_SIZE = 64
# In[]
'''make batchs of images
'''
def create_images_generator_car(df_in, batch_size, resized_shape):
    batch_image_car = []
    batch_mask_car = []
    df_in_list = (df_in).values.tolist()
#     print(len(df_in_list))
    np.random.shuffle(df_in_list)
    # return df_in_list    
    
    while True:
#         np.random.shuffle(df_in_list)
        for image_path, mask_path in df_in_list:
            image_r = cv2.imread(image_path)
            mask_r = create_mask_car(mask_path, image_r.shape)
            
            image_r = cv2.resize(image_r,(resized_shape[1], resized_shape[0]))
            mask_r = cv2.resize(mask_r,(resized_shape[1], resized_shape[0]))
            
            
            
            batch_image_car.append(image_r)
            batch_mask_car.append(mask_r)
            
            if len(batch_mask_car)>=batch_size:
                yield np.float32(np.stack((batch_image_car), 0)/255.0), np.stack(np.uint8(np.expand_dims(batch_mask_car, -1)), 0)
                batch_image_car, batch_mask_car = [], []



In [ ]:
def create_images_generator_van(df_in, batch_size, resized_shape):
    batch_image_van = []
    batch_mask_van = []
    df_in_list = (df_in).values.tolist()
#     print(len(df_in_list))
    np.random.shuffle(df_in_list)
    # return df_in_list    
    
    while True:
#         np.random.shuffle(df_in_list)
        for image_path, mask_path in df_in_list:
            image_r = cv2.imread(image_path)
            mask_r = create_mask_car(mask_path, image_r.shape)
            
            image_r = cv2.resize(image_r,(resized_shape[1], resized_shape[0]))
            mask_r = cv2.resize(mask_r,(resized_shape[1], resized_shape[0]))
            
            
            
            batch_image_van.append(image_r)
            batch_mask_van.append(mask_r)
            
            if len(batch_mask_van)>=batch_size:
                yield np.float32(np.stack((batch_image_van), 0)/255.0), np.stack(np.uint8(np.expand_dims(batch_mask_van, -1)), 0)
                batch_image_van, batch_mask_van = [], []



In [ ]:

def create_images_generator_pedestrian(df_in, batch_size, resized_shape):
    batch_image_pedestrian = []
    batch_mask_pedestrian = []
    df_in_list = (df_in).values.tolist()
#     print(len(df_in_list))
    np.random.shuffle(df_in_list)
    # return df_in_list    
    
    while True:
#         np.random.shuffle(df_in_list)
        for image_path, mask_path in df_in_list:
            image_r = cv2.imread(image_path)
            mask_r = create_mask_pedestrian(mask_path, image_r.shape)
            
            image_r = cv2.resize(image_r,(resized_shape[1], resized_shape[0]))
            mask_r = cv2.resize(mask_r,(resized_shape[1], resized_shape[0]))
            
            
            
            batch_image_pedestrian.append(image_r)
            batch_mask_pedestrian.append(mask_r)
            
            if len(batch_mask_pedestrian)>=batch_size:
                yield np.float32(np.stack((batch_image_pedestrian), 0)/255.0), np.stack(np.uint8(np.expand_dims(batch_mask_pedestrian, -1)), 0)
                batch_image_pedestrian, batch_mask_pedestrian = [], []

In [ ]:
#train_gen = create_images_generator(df_train, batch_size=BATCH_SIZE, resized_shape=resized_shape)
valid_gen_car = create_images_generator_car(df_train, batch_size=BATCH_SIZE, resized_shape=resized_shape)
val_img_car, val_mask_car = next(valid_gen_car)  

In [ ]:
val_img_car.shape

(64, 160, 256, 3)

In [ ]:
val_mask_car.shape

(64, 160, 256, 1)

In [ ]:
#train_gen = create_images_generator(df_train, batch_size=BATCH_SIZE, resized_shape=resized_shape)
valid_gen_pedestrian = create_images_generator_pedestrian(df_train, batch_size=BATCH_SIZE, resized_shape=resized_shape)
val_img_pedestrian, val_mask_pedestrian = next(valid_gen_pedestrian)  

In [ ]:
#train_gen = create_images_generator(df_train, batch_size=BATCH_SIZE, resized_shape=resized_shape)
valid_gen_van = create_images_generator_van(df_train, batch_size=BATCH_SIZE, resized_shape=resized_shape)
val_img_van, val_mask_van = next(valid_gen_van)  


In [ ]:
# load json and create model
json_file = open('output/model/model_car.json', 'r',encoding='utf-8-sig')
loaded_model_json = json_file.read()
json_file.close()
from keras.models import model_from_json
model_car = model_from_json(loaded_model_json)

# load weights into new model
model_car.load_weights("output/model/model_car.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
# load json and create model
json_file = open('output/model/model_pedestrian.json', 'r',encoding='utf-8-sig')
loaded_model_json = json_file.read()
json_file.close()
from keras.models import model_from_json
model_pedestrian = model_from_json(loaded_model_json)

# load weights into new model
model_pedestrian.load_weights("output/model/model_pedestrian.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
pred_all_car= model_car.predict(val_img_car)
np.shape(pred_all_car)

pred_all_pedestrian= model_pedestrian.predict(val_img_van)
np.shape(pred_all_pedestrian)


for i in range(64):
    print("Input: ",i)
    im = np.array(255*val_img_pedestrian[i],dtype=np.uint8)
    cv2_imshow(im)

    print("Ground Truth")
    #draw_mask_id(df_val_list[i])
    im_mask = np.array(255*val_mask_pedestrian[i],dtype=np.uint8)
    rgb_mask_true_pedestrian= cv2.cvtColor(im_mask,cv2.COLOR_GRAY2RGB)
    rgb_mask_true_pedestrian[:,:,0] = 0*rgb_mask_true_pedestrian[:,:,0]
    rgb_mask_true_pedestrian[:,:,2] = 0*rgb_mask_true_pedestrian[:,:,2]
    #img_true_pedestrian = cv2.addWeighted(rgb_mask_true_pedestrian,0.5,im,0.5,0)
    #cv2_imshow(img_true)
    im_mask = np.array(255*val_mask_car[i],dtype=np.uint8)
    rgb_mask_true_car= cv2.cvtColor(im_mask,cv2.COLOR_GRAY2RGB)
    rgb_mask_true_car[:,:,0] = 0*rgb_mask_true_car[:,:,0]
    rgb_mask_true_car[:,:,2] = 0*rgb_mask_true_car[:,:,2]
    
    img_true = cv2.addWeighted(rgb_mask_true_pedestrian,0.5,rgb_mask_true_car ,0.5,0)

    img_true = cv2.addWeighted(img_true,0.5,im,0.5,0)
    cv2_imshow(img_true)


    im_pred = np.array(255*pred_all_pedestrian[i],dtype=np.uint8)
    #rgb_mask_box = cv2.cvtColor(rgb_mask_pred, cv2.COLOR_BGR2GRAY);
    from google.colab.patches import cv2_imshow
    rgb_mask_box= im_pred
    #cv2_imshow(rgb_mask_box)
    ret,thresh = cv2.threshold(rgb_mask_box,127,255,0)
    img2 = im
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(rgb_mask_box, contours, -1, (0,255,0), 3)
    #print(contours)

    for c in contours:
        rect = cv2.boundingRect(c)
        #if rect[2] < 100 or rect[3] < 100: continue
        #print(cv2.contourArea(c))
        x,y,w,h = rect
        cv2.rectangle(img2,(x,y),(x+w,y+h),(255,0,0),3)
        cv2.putText(img2,'PEDESTRIAN',(x,y-2),0,0.3,(255,0,0))
    


    im_pred = np.array(255*pred_all_car[i],dtype=np.uint8)
    #rgb_mask_box = cv2.cvtColor(rgb_mask_pred, cv2.COLOR_BGR2GRAY);
    #from google.colab.patches import cv2_imshow
    rgb_mask_box= im_pred
    #cv2_imshow(rgb_mask_box)
    ret,thresh = cv2.threshold(rgb_mask_box,127,255,0)
    img3 = img2
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(rgb_mask_box, contours, -1, (0,255,0), 3)
    #print(contours)

    for c in contours:
        rect = cv2.boundingRect(c)
        #if rect[2] < 100 or rect[3] < 100: continue
        #print(cv2.contourArea(c))
        x,y,w,h = rect
        cv2.rectangle(img3,(x,y),(x+w,y+h),(0,255,0),3)
        cv2.putText(img3,'CAR',(x,y-2),0,0.3,(0,255,0))


    
    print("Result")
    cv2_imshow(img3)
    print("\n")
  

Output hidden; open in https://colab.research.google.com to view.